# Sentence Similarities in Seneca
In this notebook, we're going to generate sentence embeddings for sentence comparisons with a simple but effective baseline as outlined in the paper: _A Simple but Tough-to-Beat Baseline for Sentence Embeddings_ by Sanjeev Arora, Yingyu Liang, Tengyu Ma https://openreview.net/pdf?id=SyK00v5xx 

In [1]:
import os
import pickle
import re
import sys
from typing import Optional, Dict
from glob import glob
from pathlib import Path
currentdir = Path.cwd()
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir)
import numpy as np
from tqdm import tqdm
from cltk import NLP
from cltk.alphabet.lat import normalize_lat
from cltk.core.data_types import Pipeline
from cltk.embeddings import LatinEmbeddingsProcess
from cltk.languages.utils import get_lang
from cltk.sentence.lat import LatinPunktSentenceTokenizer
from cltk.tokenizers.lat.lat import LatinWordTokenizer
from cltk.tokenizers import LatinTokenizationProcess
from mlyoucanuse.text_cleaners import swallow
from scipy.spatial.distance import cosine
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
ANY_ANGLE = re.compile("<.[^>]+>") # used to remove tesserae metadata 

In [3]:
tesserae = glob(os.path.expanduser('~/cltk_data/latin/text/latin_text_tesserae/texts/*.tess'))
len(tesserae)

762

In [4]:
toker = LatinWordTokenizer()
sent_toker = LatinPunktSentenceTokenizer()

In [5]:
def toker_call(text):
    text = swallow(text, ANY_ANGLE)        
    # normalize effectively reduces our corpus diversity by 0.028%
    text = normalize_lat(text, drop_accents=True, 
                         drop_macrons=True,
                         jv_replacement=True,
                         ligature_replacement=True)
    return toker.tokenize(text)

In [6]:
latin_idf_dict_file ="word_idf.latin.pkl"

In [7]:
# to rebuild the tfidf dict, run:
# os.remove(latin_idf_dict_file)

In [8]:
if not os.path.exists(latin_idf_dict_file):
    vectorizer = TfidfVectorizer(input='filename',tokenizer=toker_call) 
    vectorizer.fit(tesserae)
    print(f"size of vocab: {len(vectorizer.vocabulary_):,}") # 299,456
    word_idf = {key: vectorizer.idf_[idx] 
                for key,idx in tqdm(vectorizer.vocabulary_.items(), total=len(vectorizer.idf_))}
    with open(latin_idf_dict_file, 'wb') as fout:
        pickle.dump(word_idf, fout)
    del vectorizer
else:
    with open(latin_idf_dict_file, 'rb') as fin:
        word_idf = pickle.load(fin) 

In [9]:
a_pipeline = Pipeline(description="A custom Latin pipeline", processes=[LatinTokenizationProcess, LatinEmbeddingsProcess], language=get_lang("lat"))
cltk_nlp = NLP(language="lat", custom_pipeline=a_pipeline)

‎𐤀 CLTK version '1.0.14'.
Pipeline for language 'Latin' (ISO: 'lat'): `LatinTokenizationProcess`, `LatinEmbeddingsProcess`.


In [29]:
# These values are needed while generating sentence embeddings
min_idf = np.min(np.array(list(word_idf.values())))
max_idf = np.max(np.array(list(word_idf.values())))
mean_idf = np.mean(np.array(list(word_idf.values())))

def rescale_idf(val):
    return (val - min_idf) / (max_idf - min_idf)

def compute_pc(X, npc=1):
    """
    Compute the principal components. DO NOT MAKE THE DATA ZERO MEAN!
    :param X: X[i,:] is a data point
    :param npc: number of principal components to remove
    :return: component_[i,:] is the i-th pc
    
    # This has been adapted from the SIF paper code: https://openreview.net/pdf?id=SyK00v5xx
    """
    svd = TruncatedSVD(n_components=npc, n_iter=7, random_state=0)
    svd.fit(X)
    return svd.components_

def remove_pc(X, npc=1):
    """
    Remove the projection on the principal components
    :param X: X[i,:] is a data point
    :param npc: number of principal components to remove
    :return: XX[i, :] is the data point after removing its projection
    
    # This has been adapted from the SIF paper code: https://openreview.net/pdf?id=SyK00v5xx
    """
    pc = compute_pc(X, npc)
    if npc==1:
        XX = X - X.dot(pc.transpose()) * pc
    else:
        XX = X - X.dot(pc.transpose()).dot(pc)
    return XX

def get_sent_embeddings(text:str, word_idf_map:Optional[Dict[str,float]]=None):
    """
    Provides the weighted average of a sentence's word vectors with the principle component removed.
    
    Expectations:
    Word can only appear once in a sentence, multiple occurrences are collapsed.
    Must have 2 or more embeddings, otherwise Principle Component cannot be found and removed.
    
    """
    cltk_doc = cltk_nlp.analyze(text=' '.join(toker.tokenize(text)))    
    if word_idf_map:
        word_idf = word_idf_map
    # else: load pickle file
    
    embed_map = {
        tmp.string: (rescale_idf(word_idf.get(tmp.string.lower(), mean_idf)), tmp.embedding)
        for tmp in cltk_doc.words
        if not np.all((tmp.embedding == 0)) # skip empty embeddings
    }    
    words = embed_map.keys()
    weights_embedds = embed_map.values()
    if len(weights_embedds) < 2: # we can't create a sentence embedding for just one word
        return np.zeros(300)    
    weights, embedds = zip(*weights_embedds)
    embedds = np.array(embedds)    
    embedds = remove_pc(embedds)    
    scale_factor = 1 / sum(weights) 
    scaled_vals = np.array([tmp * scale_factor for tmp in weights])
    # apply our weighted terms to the adjusted embeddings
    weighted_embeds = embedds * scaled_vals[:,None]
    mean_wt_embed = np.sum(weighted_embeds, axis=0)
    return mean_wt_embed

In [30]:
seneca = [tmp for tmp in tesserae if 'seneca.' in tmp ] 

In [31]:
seneca_sents = {}
for file in tqdm(seneca, total=len(seneca)):
    with open (file, 'rt') as fin:
        text = fin.read()
        text = swallow(text, ANY_ANGLE)        
        sents = sent_toker.tokenize(text)
        for idx, sent in enumerate(sents):
            text = normalize_lat(sent, drop_accents=True, 
                         drop_macrons=True,
                         jv_replacement=True,
                         ligature_replacement=True)     
            seneca_sents[text] = get_sent_embeddings(text, word_idf)

100%|██████████| 42/42 [01:42<00:00,  2.43s/it]


## Looking for similar sentences:
we'll look for sentences similar to the one that recently gave birth to this book title:

https://www.amazon.com/Dying-Every-Day-Seneca-Court/dp/0307743748/ 

* "Cotidie enim demitur aliqua pars uitae, et tunc quoque, cum crescimus, uita decrescit." 
* "Everyday we lose some part of our life, and moreover, while we grow old, our body grows weak."

In [32]:
sentences =[]  
embeddings = [] 
for key, val in seneca_sents.items():
    sentences.append(key)
    embeddings.append(val)

target_idx = [idx for idx, tmp in enumerate(sentences)
             if tmp.startswith('Cotidie enim demitur')][0]
print(f"Looking for target sentence: {sentences[target_idx]} at index: {target_idx}") 

scores = [(sentences[idx], cosine(embeddings[target_idx], embeddings[idx])) 
            for idx in range(len(embeddings))
            if not np.all((embeddings[idx] == 0)) ]

results = sorted (scores, key=lambda x: x[1])
top50 = results[:50]
print("Results:")
for sent, score in top50:
    print(f"{score:.3f}: {sent}")

Looking for target sentence: Cotidie enim demitur aliqua pars uitae, et tunc quoque, cum crescimus, uita decrescit. at index: 4874
Results:
0.000: Cotidie enim demitur aliqua pars uitae, et tunc quoque, cum crescimus, uita decrescit.
0.425: Deinde quod naturale est non decrescit mora; dolorem dies longa consumit.
0.456: Quo enim crescet, quod plenum est ?
0.506: Quomodo illa non crescit, sic ne uirtus quidem; habet numeros suos, plena est.
0.529: Pompeius non aequo laturus animo quemquam alium esse in re publica magnum et modum impositurus incrementa, quae grauia illi uidebantur, etiam cum in commune crescerent.
0.533: Quid si illi etiam nunc permiseris crescere?
0.542: Corpus enim multis eget rebus, ut ualeat; animus ex se crescit, se ipse alit, se exercet.
0.551: Idem facit ratio; non late patet, si aspicias: in opere crescit.
0.559: Haec bona non crescunt, si plena sunt.
0.567: Nam si ulla sunt, crescent et interim inpedient.
0.579: numquam meus cessabit in poenas furor
 crescetque 

Looking for target sentence: Cotidie enim demitur aliqua pars uitae, et tunc quoque, cum crescimus, uita decrescit. at index: 4874

Results:

0.000: Cotidie enim demitur aliqua pars uitae, et tunc quoque, cum crescimus, uita decrescit.

0.425: Deinde quod naturale est non decrescit mora; dolorem dies longa consumit.

0.456: Quo enim crescet, quod plenum est ?

0.506: Quomodo illa non crescit, sic ne uirtus quidem; habet numeros suos, plena est.

0.529: Pompeius non aequo laturus animo quemquam alium esse in re publica magnum et modum impositurus incrementa, 
quae grauia illi uidebantur, etiam cum in commune crescerent.

0.533: Quid si illi etiam nunc permiseris crescere?

0.542: Corpus enim multis eget rebus, ut ualeat; animus ex se crescit, se ipse alit, se exercet.

0.551: Idem facit ratio; non late patet, si aspicias: in opere crescit.

0.559: Haec bona non crescunt, si plena sunt.

0.567: Nam si ulla sunt, crescent et interim inpedient.

0.579: numquam meus cessabit in poenas furor
crescetque semper, quae ferarum immanitas,
quae Scylla, quae Charybdis Ausonium mare
Siculumque sorbens quaeue anhelantem premens
Titana tantis Aetna feruebit minis?

0.599: Deinde cum represso sanguine sicci uulneris dolor cresceret et crus suspensum equo paulatim optorpuisset, 
coactus absistere " Omnes," inquit, " iurant esse me Iouis filium, sed uulnus hoc hominem esse me clamat."

0.602: Ne itaque inuideris fratri tuo; quiescit.

0.610: Facilius enim crescit dignitas quam incipit.

0.618: Gaudium illi adferet, fiduciam confirmabit, ex conspectu mutuae tranquillitatis crescet utriusque 
laetitia.

0.622: Nec*' hoc nouum est, quaedam crescendo mutari.

0.623: Omnia enim uitia in aperto leniora sunt; morbi quoque tunc ad sanitatem inclinant, cum ex abdito erumpunt 
ac uim suam proferunt.

0.627: Non est uir fortis ac strenuus qui laborem fugit, nisi crescit illi animus ipsa rerum difficultate."

0.629: Et ut scias quemadmodum incipiant adfectus aut crescant aut efferantur, est primus motus non uoluntarius, 
quasi praeparatio adfectus et quaedam comminatio ; alter cum uoluntate non contumaci, tamquam oporteat me 
uindicari, cum laesus sim, aut oporteat hunc poenas dare, cum scelus fecerit ; tertius motus est iam impotens, 
qui non si oportet ulcisci uult, sed utique, qui rationem euicit.

0.633: paeniteat licet,
feci— uoluptas magna me inuitam subit,
et ecce crescit, derat hoc unum mihi,
spectator iste.

0.642: Saepe enim talia exempla necessitas exigit.

0.644: Non enim deminutionem malorum in bono uiro intellego, sed uacationem; nulla debent esse, non parua.

0.646: haec, quamuis auidus nec patiens morae,
deceptus totiens tangere neglegit
obliquatque oculos oraque comprimit
inclusisque famem dentibus alligat
sed tunc diuitias omne nemus suas
demittit propius pomaque desuper
insultant foliis mitia languidis
accenduntque famem, quae iubet irritas
exercere manus— has ubi protulit
et falli libuit, totus in arduum
autumnus rapitur siluaque mobilis,
instat deinde sitis non leuior fame;
qua cum percaluit sanguis et igneis
exarsit facibus, stat miser obuios
fluctus ore petens, quos profugus latex
auertit sterili deficiens uado
conantem que sequi deserit; hic bibit
altum de rapido gurgite puluerem.

0.647: An tu ad suum finem hanc euocas, in quantum potest plurimum crescere ?

0.651: Si in nostra potestate non est, an sint adfectus, ne illud quidem est, quanti sint; si ipsis permisisti 
incipere, cum causis suis crescent tantique erunt, quanti fient.

0.651: Et illa crescunt mora tantoque plus soluendum est, quanto tardius.

0.652: Et quam impatiens iuris aequi pietas Africam fuerit, cunctis apparuit; eodem enim die Scipio Africanus, 
quo uiatoris manibus fratrem abstulerat, tribuno quoque plebis priuatus intercessit.

0.652: Saepe enim in hiemem reuoluitur.

0.653: Summum enim bonum nec infringitur nec augetur; in suo modo permanet, utcumque fortuna se gessit.

0.656: Plurimum ; diuitiae enim apud sapientem uirum in seruitute sunt, apud stultum in imperio ; sapiens 
diuitiis nihil permittit, uobis diuitiae omnia ; uos, tamquam aliquis uobis aeternam possessionem earum 
promiserit, adsuescitis illis et cohaeretis, sapiens tunc maxime paupertatem meditatur, cum in mediis diuitiis 
con- stitit.

0.660: Non est, quod quisquam excusationem mentis ingratae ab infirmitate atque inopia petat et dicat : " Quid 
enim faciam et quomodo ?

0.662: Infinita scilicet cupido crescendi, cum sibi uni parum magnus uideretur.

0.666: Non est summa felicitatis nostrae in carne ponenda; bona illa sunt uera, quae ratio dat, solida ac 
sempiterna, quae cadere non possunt, ne decrescere quidem aut minui.

0.667: Quia unusquisque in eiusmodi suorum casu irascitur, putas iudicaturos homines id fieri debere quod 
faciunt; fere enim iustum quisque affectum iudicat quem adgnoscit.

0.667: Temperatus enim timor cohibet animos, adsiduus uero et acer et extrema admouens in audaciam iacentes 
excitat et omnia experiri suadet.

0.669: Uirtus enim concordi animo decreta peragit.

0.669: Sed custodienti quoque elabitur eoque citius, quo est acrior, desinit.

0.669: Soluet hunc 'questum lacrimasque nostras
sparget huc illuc agitata classis,
cum tuba iussi dare uela nautae
et simul uentis properante remo
prenderint altum fugietque litus,
quis status mentis miseris, ubi omnis
terra decrescet pelagusque crescet,
celsa cum longe latitabit Ide?

0.670: Satis praestiterit ratio, si id unum ex dolore, quod et superest et abundat, exciderit; ut quidem nullum 
omnino esse eum patiatur, nec sperandum ulli nec concupiscendum est.

0.671: Non uideo enim, quomodo non in infimum agatur e fastigio suo deiecta uirtus.

0.673: Illa enim, quae fortuita sunt, plurimum discriminis recipiunt; aestimantur enim utilitate sumentium.

0.674: Ille enim, quod est primum, scit uiuere ?

0.674: Nihil humile, nihil seruile patiatur; numquam illi necesse sit rogare suppliciter nec prosit rogasse, 
potius causae suae et prioribus factis et bonis in futurum promissis donetur.

0.675: Commota enim semel et excussa mens ei seruit quo impellitur.

0.675: Sontes enim ille inexplicabilis subit, cui difficile est modum imponere, quia paulatim surrepit et non 
desinit serpere.

0.676: Neque enim, si aliquid illi maius in quod transeat, restat, non hoc quoque in quo nascitur, secundum 
naturam est.

0.676: Non enim prohibentur opera eius omnia, sed tantum ad alios pertinentia; ipse semper in actu est, in 
effectu tunc maximus, cum illi fortuna se opposuit.

0.677: Et haec refugienda sunt, ex quibus noua occupatio multiplexque nascetur, nec accedendum eo, unde liber 
regressus non sit ; iis admouenda manus est, quorum finem aut facere aut certe sperare possis, relinquenda, quae 
latius actu procedunt nec ubi proposueris desinunt.

0.679: Et in totum inaequalis est ; modo ultra quam oportet excurrit, modo citerius debito resistit ; sibi enim 
indulget et ex libidine iudicat et audire non uult et patrocinio non relinquit locum et ea tenet quae inuasit et 
eripi sibi iudicium suum, etiam si prauum est, non sinit.

0.680: Eodem animo  beneficium debetur, quo datur, et ideo non est neclegenter dandum ; sibi enim quisque debet, 
quod a nesciente accepit ; ne tarde quidem, quia, cum omni in officio magni aestimetur dantis uoluntas, qui tarde 
fecit, diu noluit ; utique non contumeliose ; nam eum ita natura comparatum sit, ut altius iniuriae quam merita 
descendant et illa cito defluant, has tenax memoria custodiat, quid expectat, qui offendit, dum obligat ?


# Next steps
* Raw word probabilities could probably be used as a substitute for idf, YMMV.
* In the paper they mentioned that common words of negation are automatically downweighted and that accuracy could probably be boosted by _cooking_ those values appropriately. This is left as an exercise for the reader, but don't go burning down the house.